# 지도학습(4) - 회귀모델 성능 측정 지표

회귀분석을 통해 예측 모델을 만들고 해당 모델의 성능(=검정) 측정
사이킷런의 성능 측정 지표 모듈 


In [41]:
import numpy as np
import seaborn as sb
from pandas import read_excel, DataFrame
from matplotlib import pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

# 성능 측정 지표 모듈
from sklearn.metrics import r2_score, mean_absolute_error,mean_squared_error,mean_squared_log_error

In [42]:
# 자동자 주행거리 예제 불러오기

origin = read_excel("https://data.hossam.kr/E04/cars.xlsx")
origin.head()

,speed,dist
0,4,2
1,4,10
2,7,4
3,7,22
4,8,16


In [43]:
# 독립변수에 대한 다항식 생성

'''다항식 특성은 기존의 특성을 거듭제곱하거나 상호작용(interaction) 항을 생성하여 모델이 더 복잡한 데이터 패턴을 학습할 수 있도록 도와줍니다. 예를 들어, 단순한 선형 회귀 모델로는 데이터의 패턴을 잡아내기 어려운 경우가 있을 수 있는데, 이 때 다항식 특성을 추가하면 모델이 더 나은 적합성을 보일 수 있습니다.
PolynomialFeatures 클래스는 다음과 같은 매개변수를 받아서 다항식 특성을 생성합니다:
degree: 생성할 다항식의 최대 차수를 지정합니다.
interaction_only: True로 설정하면 상호작용(interaction) 항만 생성하고, 거듭제곱 항은 생성하지 않습니다.
include_bias: True로 설정하면 상수항(절편)을 포함한 특성을 생성합니다.'''

poly = PolynomialFeatures(include_bias = False)
fit = poly.fit_transform(origin[['speed']])
x = DataFrame(fit,columns=poly.get_feature_names_out())
x.head()

,speed,speed^2
0,4.0,16.0
1,4.0,16.0
2,7.0,49.0
3,7.0,49.0
4,8.0,64.0


In [44]:
# 종속변수 추출
y = origin[['dist']]
y.head()

,dist
0,2
1,10
2,4
3,22
4,16


In [45]:
# 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=123)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((35, 2), (15, 2), (35, 1), (15, 1))

In [46]:
model = LinearRegression()
fit = model.fit(x_train, y_train)

print('계수',fit.coef_)
print('절편',fit.intercept_)
print('훈련 데이터 설명력',fit.score(x_train,y_train))
print('검증 데이터 설명력',fit.score(x_test,y_test))


계수 [[0.73773403 0.09986278]]
절편 [4.70067321]
훈련 데이터 설명력 0.5941347702404498
검증 데이터 설명력 0.818350403785495


예측값 생성

In [47]:
# 훈련데이터에 대한 예측값
y_train_pred = fit.predict(x_train)
y_train_pred = y_train_pred.reshape(-1)
y_train_pred

array([ 9.24941376, 22.06429118, 42.06928848, 85.55825922, 38.23580838,
       50.33542533, 31.16802484, 54.76808209, 42.06928848, 22.06429118,
       34.60205383, 79.92724915, 14.75808748, 38.23580838, 74.49596463,
       16.99376316, 31.16802484, 54.76808209, 27.9337214 , 69.26440567,
       38.23580838, 54.76808209, 59.40046439, 24.89914351, 50.33542533,
       50.33542533, 34.60205383, 59.40046439, 34.60205383, 31.16802484,
       59.40046439, 50.33542533, 46.10249413, 14.75808748, 79.92724915])

In [48]:
# 검증 데이터에 대한 예측값
y_test_pred = fit.predict(x_test)
y_test_pred = y_test_pred.reshape(-1)
y_test_pred

array([24.89914351, 27.9337214 , 46.10249413, 79.92724915, 31.16802484,
        9.24941376, 59.40046439, 27.9337214 , 46.10249413, 22.06429118,
       34.60205383, 79.92724915, 27.9337214 , 59.40046439, 19.42916439])

### 03. 회귀분석 모델의 성능 평가

회귀분석 모델의 평가를 위한 지표는 실제 값(관측치)과 회귀 예측값의 차이를 기반으로 한다.

설명력 : 1에 가까울수록 좋음
에러율 : 0에 가까울수록 좋음

### 1) 설명력

#### $R^2$ (결정계수)

회귀분석에서 가장 많이 채택되는 설명력 값

기본적으로 모델의 학습 결과를 갖고 있는 `fit`객체의 `score()`메서드를 통해서 조회 가능

In [49]:
print('훈련 데이터 설명력:', fit.score(x_train,y_train))
print('검증 데이터 설명력:', fit.score(x_test,y_test))

훈련 데이터 설명력: 0.5941347702404498
검증 데이터 설명력: 0.818350403785495


sklearn이 제공하는 metrics 객체의 r2_score() 메서드를 통해서도 조회 할 수 있다. 이때 파라미터는 관측치와 예측치를 전달한다.

r2_score() : 실제 값과 예측 값 사이의 결정 계수(R-squared)를 계산하여 회귀 모델의 설명력을 평가

In [50]:
print("훈련 데이터 설명력: ", r2_score(y_train, y_train_pred))
print("검증 데이터 설명력: ", r2_score(y_test, y_test_pred))

훈련 데이터 설명력:  0.5941347702404498
검증 데이터 설명력:  0.818350403785495


### 2) 에러율


| 종류   | 이름                             | 한글명            | 잔차계산 | 이상치에 영향 여부 |
|------|--------------------------------|----------------|------|------------|
| MAE  | Mean Absolute Error            | 평균절대오차         | 절대값  | Yes        |
| MSE  | Mean Squared Error             | 평균제곱오차         | 제곱값  | No         |
| RMSE | Root Mean Squared Error        | 평균오차           | 제곱값  | No         |
| MAPE | Mean Absolute Percentage Error | 평균 절대 백분 오차 비율 | 절대값  | Yes        |
| MPE  | Mean Percentage Error          | 평균 비율 오차       | N/A  | Yes        |




#### 1. $MAE$ (Mean Absolute Error) : 평균절대오차

![mae_expr](res/mae_expr.png)

모델의 예측값과 실제값의 차이을 모두 더하는 개념이다. MAE는 잔차(예측이 틀린 만큼) 의 합 

절대값을 취하기 때문에 직관적으로 알 수 있는 지표다.

이상치에 영향을 받지 않는다.

MAE는 절대값을 취하는 지표이기에 실제보다 낮은 값(underperformance)인지 큰 (overperformance)값인지 알 수 없다.

![mae](res/mae.jpg)

In [51]:
# MAE는 절대값을 취하는 지표이기에 실제보다 낮은 값(underperformance)인지 큰 (overperformance)값인지 알 수 없다.
print('훈련 데이터 MAE : ',mean_absolute_error(y_train,y_train_pred))
print('검증 데이터 MAE : ',mean_absolute_error(y_test,y_test_pred))


훈련 데이터 MAE :  11.606654218318873
검증 데이터 MAE :  9.579510632066908


#### $MSE$ (Mean Squared Error) - 평균 제곱 오차

![img](res/mse_expr.png)

MAE와는 다르게 제곱을 하기 때문에 모델의 실제값과 예측값의 차이의 면적의 합이다.

제곱을 하기 때문에 특이값이 존재하면 수치가 많이 늘어난다.( = 특이치에 민감함)

MSE는 제곱 -> 사각형을 만들고 -> 그러면 값이 커져서 루트를 씌운다.  -> RMSE (평균오차)

![img](res/mse.jpg)

In [52]:
# MSE는 제곱을 하기 때문에 이상치에 민감하다. 
print('훈련 데이터 MAE : ',mean_squared_error(y_train,y_train_pred))
print('검증 데이터 MAE : ',mean_squared_error(y_test,y_test_pred))


훈련 데이터 MAE :  257.54915338176914
검증 데이터 MAE :  124.43723939078457


#### $RMSE$ (Root Mean Squared Error) : 평균 오차

![rmse](res/rmse_expr.png)

MSE를 구한 값에 루트를 씌운다.

오류 지표를 실제 값과 유사한 단위로 변환하여 해석을 쉽게 한다.

In [53]:
# MSE에 루트를 씌운다.
print('훈련 데이터 MAE : ',np.sqrt(mean_squared_error(y_train,y_train_pred)))
print('검증 데이터 MAE : ',np.sqrt(mean_squared_error(y_test,y_test_pred)))


훈련 데이터 MAE :  16.048338025533024
검증 데이터 MAE :  11.155144077544879


#### $MPE$ (Mean Percentage Error) : 평균 비율 오차

![img](res/mpe_expr.png)

MAPE와 비슷하지만 MAPE에서 절대값을 제외한 지표다.

MPE : 잔차를 가지고 직접 만들어내야함. 

장점은 모델이 실제값보다 낮은 값인지 큰 값인지 판단 할 수 있다.

![img](res/mpe.jpg)

In [54]:
# API로 제공되는 기능이 아니고, 직접 계산해야 하기 때문에 관측치와 예측치의 데이터 타입이 일치해야 한다.
# -> numpy 배열 혹은 Series 타입으로 통일해야 한다.
print("훈련 데이터 MPE: ", np.mean((y_train.values - y_train_pred) / y_train.values) * 100)
print("검증 데이터 MPE: ", np.mean((y_test.values - y_test_pred) / y_test.values) * 100)

훈련 데이터 MPE:  -60.125435326146004
검증 데이터 MPE:  -157.06332774885408


# 04. 머신러닝 회귀분석 모듈 테스트

회귀분석 수행

In [55]:
import sys
sys.path.append("../../")
import helper


In [56]:
olsResult = helper.ml_ols(origin, xnames='speed', yname='dist', degree=2, test_size=0.3, random_state=123)

print('계수 :',olsResult.coef)
print('절편 :',olsResult.intercept)

계수 : [[0.73773403 0.09986278]]
절편 : [4.70067321]


훈련데이터에 대한 평가지표

In [57]:
print('R^2 :',olsResult.trainRegMetric.r2)
print('MAE :',olsResult.trainRegMetric.mae)
print('MSE :',olsResult.trainRegMetric.mse)
print('RMSE :',olsResult.trainRegMetric.rmse)
print('MAPE :',olsResult.trainRegMetric.mape)
print('MPE :',olsResult.trainRegMetric.mpe)

R^2 : 0.5941347702404498
MAE : 11.606654218318873
MSE : 257.54915338176914
RMSE : 16.048338025533024
MAPE : 33.730257586807205
MPE : -16.22850108764364


검증 데이터에 대한 평가지표

In [58]:
print('R^2 :',olsResult.testRegMetric.r2)
print('MAE :',olsResult.testRegMetric.mae)
print('MSE :',olsResult.testRegMetric.mse)
print('RMSE :',olsResult.testRegMetric.rmse)
print('MAPE :',olsResult.testRegMetric.mape)
print('MPE :',olsResult.testRegMetric.mpe)

R^2 : 0.818350403785495
MAE : 9.579510632066908
MSE : 124.43723939078457
RMSE : 11.155144077544879
MAPE : 53.57568788002909
MPE : -32.8120697063403
